In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Underweight/GSE131835'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Differential gene expression between adipsoe depots in cancer cachexia"
!Series_summary	"Cancer cachexia (CC) is a poorly understood cause of morbidity and mortality, which has no efficacious treatment or generally-accepted management strategy. The consensus definition for CC identified skeletal muscle loss as a key marker in the diagnosis and classification of cachexia. The importance of fat wasting however, is less understood. During cachexia, different adipose depots demonstrate differential wasting rates. Studies from animal models have suggested adipose tissue may be a key driver of muscle wasting through fat-muscle crosstalk but human studies in this area are lacking. We performed global gene expression profiling of intra-abdominal (IAAT) and subcutaneous (SAT) adipose from weight stable and cachectic cancer patients and healthy controls."
!Series_summary	"Cachexia was defined as >2% weight loss plus low CT-muscularity. Biopsies of SAT and I

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine gene expression data availability
is_gene_available = True  # Given the description, it seems to be gene expression data

# Determine the availability and keys for 'Underweight', 'age', 'gender'
# For 'Underweight' (approximated as 'group' in this context)
#   Weight loss info in cachexia is complex but let's treat 'group' as a proxy for now
trait_row = 1  # Based on groups: CWS, CWL, CONTROL, CONTROl

# For 'age'
age_row = 3  # ages are explicitly listed under key 3

# For 'gender'
gender_row = 2  # genders are explicitly listed under key 2

# Data type conversions

# Conversion functions
def convert_trait(value):
    value = value.split(':')[1].strip().upper()
    if value in ["CWS", "CWL"]:
        return 1  # Presuming these to represent Underweight (cachexia-related)
    elif "CONTROL" in value:
        return 0  # Control group, assumed not underweight
    else:
        return None

def convert_age(value):
    try:
        age = int(value.split(':')[1].strip())
        return age
    except:
        return None

def convert_gender(value):
    gender = value.split(':')[1].strip().lower()
    if gender == "male":
        return 1
    elif gender == "female":
        return 0
    else:
        return None

# Save cohort information
save_cohort_info('GSE131835', './preprocessed/Underweight/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Underweight', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Underweight/trait_data/GSE131835.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM3821249': [1, 51, 1], 'GSM3821251': [1, 64, 1], 'GSM3821252': [1, 62, 0], 'GSM3821254': [1, 78, 0], 'GSM3821255': [0, 47, 1], 'GSM3821257': [0, 59, 0], 'GSM3821258': [1, 51, 1], 'GSM3821259': [1, 57, 1], 'GSM3821261': [0, 47, 1], 'GSM3821262': [1, 58, 1], 'GSM3821264': [0, 53, 0], 'GSM3821265': [1, 78, 0], 'GSM3821267': [1, 62, 0], 'GSM3821268': [1, 64, 1], 'GSM3821269': [0, 49, 0], 'GSM3821271': [0, 49, 0], 'GSM3821273': [1, 58, 1], 'GSM3821274': [1, 57, 1], 'GSM3821276': [0, 59, 0], 'GSM3821277': [0, 54, 1], 'GSM3821278': [0, 54, 1], 'GSM3821280': [0, 53, 0], 'GSM3821281': [0, 60, 1], 'GSM3821283': [1, 56, 0], 'GSM3821284': [1, 64, 1], 'GSM3821285': [0, 41, 0], 'GSM3821287': [0, 41, 0], 'GSM3821288': [1, 56, 0], 'GSM3821290': [0, 60, 1], 'GSM3821291': [1, 56, 0], 'GSM3821292': [1, 56, 0], 'GSM3821293': [1, 76, 1], 'GSM3821295': [1, 76, 1], 'GSM3821296': [1, 81, 1], 'GSM3821298': [0, 48, 1], 'GSM3821299': [0, 48, 1], 'GSM3821301': [1, 65, 0], 'GSM3821302': [1, 68, 1], 'GSM3821304

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ENSG00000000003_at', 'ENSG00000000005_at', 'ENSG00000000419_at',
       'ENSG00000000457_at', 'ENSG00000000460_at', 'ENSG00000000938_at',
       'ENSG00000000971_at', 'ENSG00000001036_at', 'ENSG00000001084_at',
       'ENSG00000001167_at', 'ENSG00000001460_at', 'ENSG00000001461_at',
       'ENSG00000001497_at', 'ENSG00000001561_at', 'ENSG00000001617_at',
       'ENSG00000001626_at', 'ENSG00000001629_at', 'ENSG00000001631_at',
       'ENSG00000002016_at', 'ENSG00000002330_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ENSG00000000003_at', 'ENSG00000000005_at', 'ENSG00000000419_at', 'ENSG00000000457_at', 'ENSG00000000460_at'], 'Ensembl Gene ID': ['ENSG00000000003', 'ENSG00000000005', 'ENSG00000000419', 'ENSG00000000457', 'ENSG00000000460'], 'HGNC ID': ['HGNC:11858', 'HGNC:17757', 'HGNC:3005', 'HGNC:19285', 'HGNC:25565'], 'ENTREZ_GENE_ID': [7105.0, 64102.0, 8813.0, 57147.0, 55732.0], 'ORF': ['TSPAN6', 'TNMD', 'DPM1', 'SCYL3', 'C1orf112'], 'SPOT_ID': ['ENSG00000000003', 'ENSG00000000005', 'ENSG00000000419', 'ENSG00000000457', 'ENSG00000000460']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Decide keys for identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'ORF'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function.
gene_mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library, and update gene_data.
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Underweight/gene_data/GSE131835.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data are severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Underweight')

# 4. Save the cohort information.
save_cohort_info('GSE131835', './preprocessed/Underweight/cohort_info.json', True, True, trait_biased, merged_data)

# 5. If the trait is not severely biased, save the merged data to a csv file.
if not trait_biased:
    csv_path = './preprocessed/Underweight/GSE131835.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Underweight', the least common label is '0.0' with 16 occurrences. This represents 33.33% of the dataset.
The distribution of the feature 'Underweight' in this dataset is fine.

Quartiles for 'Age':
  25%: 53.75
  50% (Median): 59.0
  75%: 65.0
Min: 41.0
Max: 81.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 18 occurrences. This represents 37.50% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

